In [15]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Alpha-Beta Pruning


In [16]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [17]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

## AlphaBetaMax

In [18]:
function alphaBetaMax(State, alpha, beta, score, depth)
    if isterminal(State) 
        return terminal_evaluation(State) - depth
    end
    for move in moves(State)
        if depth == 0
            value = evaluate_move(State, move, score)
        else
            value = alphaBetaMin(domove(State, move), alpha, beta, evaluate_move(State, move, score), depth - 1)
        end
        if value >= beta
            return value
        end
        alpha = max(alpha, value)
    end
    return alpha
end

alphaBetaMax (generic function with 1 method)

## AlphaBetaMin

In [21]:
function alphaBetaMin(State, alpha, beta, score, depth)
    if isterminal(State) 
        return terminal_evaluation(State) + depth
    end
    for move in moves(State)
        if depth == 0
            value = evaluate_move(State, move, score)
        else
            value = alphaBetaMax(domove(State, move), alpha, beta, evaluate_move(State, move, score), depth - 1)
        end
        if value <= alpha
            return value
        end
        beta = min(beta, value)
    end
    return beta
end

alphaBetaMin (generic function with 1 method)

In [22]:
b = fromfen("7k/P7/8/8/8/8/8/K7 w - - 0 1")
display(b)
alphaBetaMax(b, -Inf, Inf, evaluate_position(b), 5)

Board (7k/P7/8/8/8/8/8/K7 w - -):
 -  -  -  -  -  -  -  k 
 P  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 K  -  -  -  -  -  -  -

885.0

## Alpha-beta Pruning function

In [25]:
function alphaBetaPruning(State, score, depth)
    next_moves = moves(State)
    if sidetomove(State) == WHITE
        bestVal = alphaBetaMax(State, -Inf, Inf, score, depth)
        BestMoves = [move for move in next_moves if alphaBetaMin(domove(State, move), -Inf, Inf, evaluate_move(State, move, score), depth - 1) == bestVal]
    elseif sidetomove(State) == BLACK
        bestVal = alphaBetaMin(State, -Inf, Inf, score, depth)
        BestMoves = [move for move in next_moves if alphaBetaMax(domove(State, move), -Inf, Inf, evaluate_move(State, move, score), depth - 1) == bestVal]
    end
    BestMove = rand(BestMoves)
    return bestVal, BestMove
end

alphaBetaPruning (generic function with 1 method)